In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from selenium import webdriver
import time
from random import random

In [2]:
# Scraping "free-proxy-list.net" to get a list of proxies.
res = requests.get("https://free-proxy-list.net")
proxies = pd.read_html(res.text)
proxies = proxies[0][:-1]
proxies.columns = [c.replace(' ', '_') for c in proxies.columns]
ip = list(proxies.IP_Address)
port = list(map(str, list(map(int, list(proxies.Port)))))
print(ip)
print(port)

['114.134.190.230', '187.94.31.237', '43.248.24.158', '45.70.63.93', '202.57.35.74', '216.228.69.202', '150.129.52.74', '197.211.238.220', '140.227.205.216', '175.100.72.95', '185.100.13.5', '190.107.20.206', '114.110.21.50', '103.86.187.242', '197.254.14.70', '113.160.206.37', '109.172.57.250', '67.60.137.219', '84.53.238.49', '176.36.3.23', '132.255.90.20', '171.6.150.33', '1.10.188.140', '103.60.187.1', '159.224.226.164', '62.99.178.46', '171.96.56.194', '202.21.103.234', '103.21.160.10', '125.25.80.39', '118.172.51.110', '188.169.123.54', '167.71.123.107', '159.192.97.181', '110.74.221.18', '181.129.66.138', '190.166.249.44', '94.73.234.169', '190.152.71.230', '170.82.231.26', '194.58.182.73', '1.20.103.196', '212.227.11.130', '185.46.223.198', '114.134.186.59', '195.191.250.38', '88.87.90.107', '188.115.191.192', '41.139.9.47', '202.143.120.25', '36.90.26.2', '188.240.110.228', '83.10.49.60', '138.36.228.142', '177.185.158.78', '79.104.8.150', '41.60.233.107', '92.112.25.185', '11

In [5]:
def conf_webdriver(proxy_id, ip, port):    # Configuring the Proxy in Firefox webdriver.
    PROXY = ip[proxy_id] + ":" + port[proxy_id]
    webdriver.DesiredCapabilities.FIREFOX['proxy']={
        "sslProxy":PROXY,
        "ftpProxy":PROXY,
        "socksProxy":PROXY,
        "socksVersion":5,
        "proxyType":"MANUAL"
    }
    print("Change proxy: " + PROXY)

# Preparing the variables that will contain the data.
category_data = []
localite_data = []
criterion_data = []
price_data = []
monthly_data = []
description_data = []
plus_data = []
energie_data = []

conf_webdriver(0, ip, port)

max = 20    # The maximum seconds of delay.
min = 11    # The minimum seconds of delay.

page_num1=155  # Current page number.
page_num2=page_num1+2   # Modify the number of pages to scrape.

# Loop the pages.
proxy_id = 10
count_page = 0
for num in range(page_num1,page_num2):   
    driver = webdriver.Firefox()    # Open Firefox webdriver.
    
    # ----------- Testing the availability of a Proxy. -------------
    cond = 0
    while(cond == 0):   
        try:
            driver.get("https://seloger.com/list.htm?projects=2%2C5&types=1%2C2%2C4%2C3%2C9%2C13%2C14%2C12%2C11%2C10&natures=1%2C2%2C4&places=%5B%7BidPays%3A250%7D%5D&enterprise=0&houseboat=1&qsVersion=1.0&LISTING-LISTpg="+str(num))
            cond = 1
        except:
            driver.quit()
            conf_webdriver(proxy_id, ip, port)
            driver = webdriver.Firefox()
            proxy_id = proxy_id+1
    # -------------------------------------------------------------        
    
    # ----- Adding a random delay. -----
    time_random1 = random()
    time1 = min + (time_random1 * (max - min))
    time.sleep(time1)
    # ---------------------------
    
    url2 = str(driver.current_url)   # Get the current url.
    
    # ------ When seloger.com find out that you are a bot, we will change the page and the Proxy. ------
    if('validate.perfdrive.com' in url2):  
        driver.quit()
        proxy_id +=1 
        conf_webdriver(proxy_id, ip, port)
        driver = webdriver.Firefox()
        continue
    # ----------------------------------------------------------------------------------------------
    
    page = soup(driver.page_source, 'html.parser')  # Get the HTML code of the current page.
    house_containers = page.find_all("div", class_="block__ShadowedBlock-sc-10w6hsj-0 ListContent__SmartClassifiedExtended-sc-1viyr2k-2 iddbNe classified__ClassifiedContainer-sc-1wmlctl-0 jzFgwH Card__CardContainer-sc-7insep-8 hglOTD")
    print("Number of ads: " + str(len(house_containers)))  # The number of ads in a single page (25).
    
    driver2 = webdriver.Firefox()   # Open a second Firefox webdriver.
    proxy_id2 = 10
    
    # Loop the ads.
    for data in house_containers:
        find_link = data.find_all("a", class_="CoveringLink-a3s3kt-0 dXJclF")    # Getting the ad link.
        L=str(find_link[0]).split(" ")
        ad_link=L[3][14:-1]
               
        if ('seloger.com' not in ad_link):  # If the ad isn't in seloger.com, we will skip it.
            continue
        
        # ----------- Testing the availability of a new Proxy. -------------
        cond2 = 0
        while(cond2 == 0):  
            try:
                driver2.get("https://" + ad_link)
                cond2 = 1
            except:
                driver2.quit()
                conf_webdriver(proxy_id2, ip, port)
                driver2 = webdriver.Firefox()
                proxy_id2 = proxy_id2+1
        # -----------------------------------------------------------------
        
        # ----- Adding a random delay. -----
        time_random2 = random()
        time2 = min + (time_random2 * (max - min))
        time.sleep(time2)
        # ---------------------------
        
        url = str(driver2.current_url)     # Get the current url.
       
        # ------ When seloger.com find out that you are a bot, we will change the ad and the Proxy. ------
        if('validate.perfdrive.com' in url): 
            driver2.quit()
            proxy_id2 +=1 
            conf_webdriver(proxy_id2, ip, port)
            driver2 = webdriver.Firefox()     
            continue
        # -----------------------------------------------------------------------------------------------
            
        page2 = soup(driver2.page_source, 'html.parser')  # Get the HTML code of the ad.
        
        criterion = []
        plus = []
        energie =[]
        
        # Getting the category.
        try:   
            try:    
                try:    # The first version of seloger.com.
                    find_category = page2.find_all("h2", class_="c-h2")[0]
                    category = str(find_category)[len('<h2 class="c-h2">'):-5]
                except IndexError:   # The second version of seloger.com.
                    find_category = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[0]
                    category=(str(find_category)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:   # The third version of seloger.com.
                find_category = page2.find_all("div", class_="Summarystyled__Title-tzuaot-3 dkepid")[0]
                category=(str(find_category)[len('<div class="Summarystyled__Title-tzuaot-3 dkepid">'):-6])
        except:   # When the category doesn't exist.
            category = "NULL"
            
        # Getting the City.
        try:
            try:
                try:
                    find_localite = page2.find_all("p", class_="localite")[0]
                    localite = str(find_localite)[len('<p class="localite">'):-4]
                except IndexError:
                    find_localite = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[1]
                    localite=(str(find_localite)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:
                find_localite = page2.find_all("div", class_="Summarystyled__Address-tzuaot-5 fLUFm")[0]
                localite=(str(find_localite)[len('<div class="Summarystyled__Address-tzuaot-5 fLUFm">'):-6])
        except:
            localite = "NULL"
            
        # Getting the criterion (There are only two versions of seloger.com for the criterion).
        try:
            try:
                find_criterion = page2.find_all("ul", class_="criterion")[0]
                criterion2 = str(find_criterion).split('<li>')[1:]
                for i in range (len(criterion2)):
                    ch=""
                    for j in range (len(criterion2[i])):
                        if(str(criterion2[i])[j]=="<"):
                            break
                        ch=ch+str(criterion2[i])[j]
                    criterion.append(ch)

            except IndexError:
                find_criterion = page2.find_all("div", class_="Tags__TagContainer-sc-1gccxml-0 kQEoGa")[:3]
                find_criterion = list(map(str, find_criterion))
                for i in range (len(find_criterion)):
                    criterion.append(find_criterion[i][len('<div class="Tags__TagContainer-sc-1gccxml-0 kQEoGa">'):-6])

        except:
            criterion.append("NULL")
            
        # Getting the price.
        try:
            try:
                try:
                    find_price = page2.find_all("a", class_="js-smooth-scroll-link price")[0]
                    price = str(find_price)[len('<a class="js-smooth-scroll-link price" data-smoothscroll-offset="160" href="#about_price_anchor">'):-4].replace(" ","").replace("\n","")
                except IndexError:
                    find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[0]
                    price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[0]
                price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            price = "NULL"
            
        # Getting the monthly price.
        try:
            try:
                try:
                    find_monthly = page2.find_all("a", class_="monthly")[0]
                    monthly = str(find_monthly)[len('<a class="monthly" href="https://e-immobilier.credit-agricole.fr/simulca/?ORI=selogercampagne&amp;xtor=AL-2-[partenariat]-[Seloger]-[pdd]-[lien_a_partir_sticky]" rel="nofollow" target="_blank">'):-len('<sup>1</sup></a>')]
                except IndexError:
                    find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[1]
                    monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[1]
                monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            monthly = "NULL"

        # Getting the description.    
        try:
            try:
                find_description = page2.find_all("p", class_="sh-text-light")[0]
                description = str(find_description)[len('<p class="sh-text-light" id="js-descriptifBien">'):-4]
            except IndexError:
                find_description = page2.find_all("div", class_="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd")[0]
                description = str(find_description)[len('<div class="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd">'):-len('<button class="ContactLink__StyledButton-sc-1ex39dt-0 jPagdq">Demander plus d\'informations à l\'agence<div class="ContactLink__Chevron-sc-1ex39dt-1 gYbUwK Icon__IconElement-sc-1xnn3zd-0 fFMQXr" color="#e00034" data-test="contact-link-chevron" style="margin-left:5px"><svg viewbox="0 0 512 512"><path d="M144.9 495.9c-9 0-16.4-7.3-16.4-16.4 0-4.2 1.6-8.3 4.6-11.4L338 256 133.1 43.8c-6.2-6.5-6-16.9.5-23.2 6.5-6.2 16.8-6 23 .4l226.9 235-226.8 234.9c-3.1 3.2-7.4 5-11.8 5z"></path></svg></div></button>')]
        except:
            description = "NULL"
        
        # Getting the "more information".
        try:
            try:
                find_plus = page2.find_all("div", class_="u-left")[0]
                find_plus = page2.find_all("div", class_="u-left")
                for i in range (len(find_plus)):
                    plus2 = str(find_plus[i]).replace('<div class="u-left">', "").replace("</div>", "")
                    plus.append(plus2)

            except IndexError:
                find_plus = page2.find_all("figcaption") + page2.find_all("li", class_="GeneralList__Item-sc-9gtpjm-1 dsevYN")
                for i in range (len(find_plus)):
                    ch=""
                    for j in range (len(str(find_plus[i]))):
                        if(str(find_plus[i])[j]=='>'):
                            k=j
                            while(str(find_plus[i])[k]!="<"):
                                ch=ch + str(find_plus[i])[k]
                                k=k+1
                            break
                    plus.append(ch[1:])
        except:
            plus.append("NULL")

        # Getting the energy information
        try:
            try:
                find_energie = page2.find_all("div", class_="info-detail")[0]
                find_energie = page2.find_all("div", class_="info-detail")

                energie.append(str(find_energie[0])[len('<div class="info-detail">'):-6])
                energie.append(str(find_energie[1])[len('<div class="info-detail">'):-6])

            except IndexError:
                find_energie = page2.find_all("span", class_="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO")
                energie.append(str(find_energie[0])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
                energie.append(str(find_energie[1])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
        except:
            energie.append("NULL")
            energie.append("NULL")


        print("Category: " + str(category))
        print("City: " + str(localite))
        print("Criterion: " + str(criterion))
        print("Price: " + str(price))
        print("Monthly: " + str(monthly))
        print("Description: " + str(description))
        print("More information: " + str(plus))
        print("Energy: " + str(energie))
        print("Page Number: " + str(num))
        print("-----------------------------------------------------")

        category_data.append(str(category))
        localite_data.append(str(localite))
        criterion_data.append(criterion)
        price_data.append(str(price))
        monthly_data.append(str(monthly))
        description_data.append(str(description))
        plus_data.append(plus)
        energie_data.append(energie)
    
    # Cleaning the criterion field
    for i in range (len(criterion_data)):
        if (len(criterion_data[i])==1):
            if("pièce" in criterion_data[i][0]):
                criterion_data[i].insert(1, "NULL")
                criterion_data[i].insert(2, "NULL")
            elif("chambre" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(2, "NULL")
            elif("m²" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(1, "NULL")
            else:
                criterion_data[i][0] = "NULL"
                criterion_data[i].append("NULL")
                criterion_data[i].append("NULL")
                
        if (len(criterion_data[i])==2):
            if(("pièce" in criterion_data[i][0]) and ("chambre" in criterion_data[i][1])):
                criterion_data[i].insert(2, "NULL")
            elif(("pièce" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(1, "NULL")
            elif(("chambre" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(0, "NULL")
            else:
                criterion_data[i][0] = "NULL"
                criterion_data[i][1] = "NULL"
                criterion_data[i].append("NULL")
                
        if(len(criterion_data[i])==0):
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
    
    # Saving the data.
    df = pd.DataFrame()
    df["category"] = category_data
    df["localite"] = localite_data
    df["piece"] = [i[0] for i in criterion_data]
    df["chambres"] = [i[1] for i in criterion_data]
    df["area"] = [i[2] for i in criterion_data]
    df["price"] = price_data
    df["monthly"] = monthly_data
    df["description"] = description_data
    df["Diagnostic de performance énergétique(kWhEP/m².an)"] = [i[0] for i in energie_data]
    df["Indice d'émission de gaz à effet de serre(kgeqCO2/m².an)"] = [i[1] for i in energie_data]
    df["plus"] = plus_data
    df["type"] = "Acheter"
    
    df.to_csv("./page.csv", index=False)
    csv1 = pd.read_csv("./page.csv")
    csv2 = pd.read_csv("./full_data.csv")
    full_data = pd.concat([csv1, csv2])
    full_data.to_csv("./full_data.csv", index=False)
    print('**DONE**')
    
    # After 3 successful scrape, we will reload the proxy list.
    count_page += 1
    if(count_page>=3):
        count_page = 0
        proxy_id = 10

Change proxy: 114.134.190.230:37294
Change proxy: 185.100.13.5:59728
Change proxy: 190.107.20.206:34626
Change proxy: 114.110.21.50:50464
Change proxy: 103.86.187.242:23500
Change proxy: 197.254.14.70:37941
Change proxy: 113.160.206.37:37194
Change proxy: 109.172.57.250:23500
Number of ads: 25
Category: Maison / Villa
City: Bouliac
Criterion: ['9 pièces', '280 m²', '9 pièces']
Price: 1 196 000 €
Monthly: 4 937 €
Description: style="max-height: 66px;"><p>Cette maison d'exception de 288 m², réalisée par architecte, a été construite en 2009 et bâtie dans le quartier le plus recherché de Bouliac, célèbre village de la rive droite surplombant la Garonne. Dès l'entrée, la maison offre une magnifique pièce de vie de 120 m² avec de belles hauteurs sous plafond et une cuisine ouverte entièrement équipée. L'ensemble s'ouvre par de larges baies vitrées sur l'extérieur offrant une terrasse et une piscine chauffée de 5m X 10m, surplombant le terrain verdoyant de 2400 m². Un couloir dessert deux cha

Category: Maison / Villa
City: Villers-sous-Saint-Leu
Criterion: ['6 pièces', '140 m²', '5 pièces']
Price: 325 000 €
Monthly: 1 348 €
Description: style="max-height: 66px;"><p>Gouvieux PROCHE joli pavillon vous offrant: entrée, salon/séjour avec cheminée insert, cuisine équipée américaine, buanderie, WC, une chambre et une belle salle d'eau. A l'étage: 3 chambres dont 1 suite parentale avec douche à l'italienne, dressing, un bureau et WC. Joli jardin clos d'environ 443 m² et un garage. Votre agence NESTENN de Lamorlaye vous accueille du lundi au samedi de 9h à 12h et de 14h à 19h. Vous pouvez nous contacter Retrouvez nos annonces sur.</p><butto
More information: ['Cheminée', 'Surface de 139m²', '7 Pièces', '5 Chambres', 'Année de construction 1990', "2 Salles d'eau", '2 Toilettes', 'Chauffage individuel électrique radiateur', 'Cuisine américaine équipée', 'Une entrée', 'Salle de séjour : 25m²', 'Terrain de 443m²', '1 Parking', '1 Box']
Energy: ['264', '14']
Page Number: 136
-----------

Category: Maison / Villa
City: La Rochelle
Criterion: ['6 pièces', '137 m²', '5 pièces']
Price: 477 000 €
Monthly: 1 975 €
Description: style="max-height: 66px;"><p>Stéphane Grandener A 12 Mn de La Rochelle, secteur Saint Xandre maison de plain pied édifiée en 2001 en excellent état avec piscine intérieure chauffée comprenant entrée avec placard,, 2 WC, pièce de vie avec cheminée, cuisine aménagée et équipée, cellier, salle de bains + douche, 4 chambres avec placard dont une avec salle d'eau. Double garage de 50 m² avec porte automatique, une cave. Pièce piscine de 62 m² climatisée. La maison est équipée d'un chauffage central, de baies à galandage, de fenêtres oscillo-battantes. Terrain d'environ 950 m² profitant d'une belle exposition et offrant une vue dégagée. Possibilité d'acquérir un type 2 de 50 m² attenant à la maison avec accès indépendant. Dpe C (477 000 euros dont honoraires 4 % à la charge acquéreur soit 459 000 euros hors honoraires) réf 609.</p><butto
More information: ['

Category: Appartement
City: Quartier <!-- -->Dupleix-Motte Picquet<!-- -->, <!-- -->Paris 15ème
Criterion: ['2 pièces', '57.89 m²', '3 pièces']
Price: 728 000 €
Monthly: 3 009 €
Description: style="max-height: 66px;"><p>Paris 15 Nord / Limite 7ème - Quartier SUFFREN-Tour Eiffel. Dans un bel immeuble haussmannien de standing.(Rénovation parties comunes votées et payées). Au 2ème étage, un appartement 2/3 pièces décoré par architècte de 56.02m². Comprenant: une entrée, un séjour, une cuisine américaine aménagée et équipée, un séjour double exposé sud-ouest (une partie salon, une partie salle à manger), une chambre au calme absolu sur cour et jardin, une salle de bains, WC indépendant, un dressing. L'appartement est proposé intégralement meublé. L'ensemble donne sur une jolie cour arborée, exposé sud-ouest. En parfait état. Une CAVE complètent ce bien. Loi ALUR: Copropriété incluant 17 lots, Quote-part annuelle du budget prévisionnel d'environ 1 300 € / an - soit 108 € / mois. Aucune proc

Category: Appartement
City: Quartier <!-- -->Centre Ville<!-- -->, <!-- -->Reims
Criterion: ['1 pièce', '21.34 m²', '1 pièce']
Price: 106 700 €
Monthly: 449 €
Description: style="max-height: 66px;"><p>HYPER CENTRE - Cet appartement de type 1 bis d'une superficie de 25.93 m², comprend une pièce de vie, une cuisine aménagée et équipée ainsi qu'une salle d'eau avec WC. Au sein d'une petite copropriété à faibles charges, l'appartement dispose également d'une cave. Au coeur du centre ville et à deux pas du tramway, ce bien est à proximité de toutes les commodités (commerces, écoles, transports,..) Idéalement situé pour étudiants ou investissement locatif. A visiter avec Stéphane Plaza. ESTIMATIONS OFFERTES - STÉPHANE PLAZA IMMOBILIER REIMS - 4-6 Place de l'Hôtel de Ville - 51100 REIMS - Notre équipe est à votre disposition du Lundi au Vendredi de 9h00 à 12h00 et de 14h00 à 19h00 - Le samedi de 9h à 12h et de 14h à 17h. Tél: Mail: Copropriété de 24 lots (Pas de procédure en cours). Charges a

Category: Appartement
City: Quartier <!-- -->Centre Montmorency Centre<!-- -->, <!-- -->Montmorency
Criterion: ['3 pièces', '75 m²', '4 pièces']
Price: 295 000 €
Monthly: 1 225 €
Description: style="max-height: 66px;"><p>A vendre appartement 4 pièces à Montmorency. Située en centre ville, dans résidence de standing et au calme, bel appartement de type F4 avec 2 balcons. Offrant: un dégagement, une entrée, un séjour double, une cuisine, WC, une salle d'eau et 2 chambres. Une place de parking en sous-sol, une place de parking extérieur et une cave s'ajoutent à ce bien qui est à voir sans tarder. Dont 4.98 % honoraires TTC à la charge de l'acquéreur. Copropriété de 52 lots (Pas de procédure en cours). Charges annuelles: 3720.00 euros.</p><butto
More information: ['Cave', '2 Balcons', 'Gardien', 'Ascenseur', 'Surface de 75,75m²', 'Bâtiment de 3 étages', '4 Pièces', '2 Chambres', 'Année de construction 1977', 'Au 1er étage', "1 Salle d'eau", '1 Toilette', 'Chauffage central radiateur', 'Cui

In [14]:
driver.quit()
driver2.quit()

In [19]:
full_data = pd.read_csv("./full_data.csv")
print(full_data.shape)

(1614, 12)


In [25]:
# Delete duplicates.
full_data.drop_duplicates(keep='first', inplace=True)
full_data.to_csv("./full_data2.csv", index=False)
print(full_data.shape)

(1776, 12)
